In [1]:
import os
from pyspark.find_spark_home import _find_spark_home
from pyspark.sql import SparkSession
from pyspark import StorageLevel, SparkConf, SparkContext
from pyspark.sql.functions import mean
from pyspark.sql.types import IntegerType, FloatType

from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression
from pyspark.sql.functions import col
from pyspark.sql.functions import lit

print(_find_spark_home())

/home/lyu0001/anaconda3/envs/BigData/lib/python3.11/site-packages/pyspark


In [2]:
#pip install --upgrade pandas

In [3]:
conf = (
    SparkConf()
    .setMaster("local[*]")
    .set("spark.local.dir", "/home/lyu0001/spark-temp")
    .set("spark.driver.memory", "6g")
    .set("spark.executor.memory", "12g")
    .set("spark.driver.maxResultSize", "10g")
    .set("spark.pyspark.python", "/home/lyu0001/anaconda3/bin/python")
    .set("spark.pyspark.driver.python", "/home/lyu0001/anaconda3/bin/python")
)

spark = (
    SparkSession.builder.appName("PySpark Hadoop Integration")
    .config(conf=conf)
    .getOrCreate()
)
sc = spark.sparkContext

for item in sc.getConf().getAll():
    print(item)

23/12/21 02:16:52 WARN Utils: Your hostname, ubuntu resolves to a loopback address: 127.0.1.1; using 192.168.138.133 instead (on interface ens33)
23/12/21 02:16:52 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/12/21 02:16:54 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/12/21 02:16:54 WARN SparkConf: Note that spark.local.dir will be overridden by the value set by the cluster manager (via SPARK_LOCAL_DIRS in mesos/standalone/kubernetes and LOCAL_DIRS in YARN).


('spark.app.startTime', '1703153813839')
('spark.driver.host', '192.168.138.133')
('spark.executor.id', 'driver')
('spark.app.id', 'local-1703153816657')
('spark.pyspark.python', '/home/lyu0001/anaconda3/bin/python')
('spark.executor.memory', '12g')
('spark.pyspark.driver.python', '/home/lyu0001/anaconda3/bin/python')
('spark.driver.extraJavaOptions', '-Djava.net.preferIPv6Addresses=false -XX:+IgnoreUnrecognizedVMOptions --add-opens=java.base/java.lang=ALL-UNNAMED --add-opens=java.base/java.lang.invoke=ALL-UNNAMED --add-opens=java.base/java.lang.reflect=ALL-UNNAMED --add-opens=java.base/java.io=ALL-UNNAMED --add-opens=java.base/java.net=ALL-UNNAMED --add-opens=java.base/java.nio=ALL-UNNAMED --add-opens=java.base/java.util=ALL-UNNAMED --add-opens=java.base/java.util.concurrent=ALL-UNNAMED --add-opens=java.base/java.util.concurrent.atomic=ALL-UNNAMED --add-opens=java.base/sun.nio.ch=ALL-UNNAMED --add-opens=java.base/sun.nio.cs=ALL-UNNAMED --add-opens=java.base/sun.security.action=ALL-UNN

In [5]:
#展示数据
df = spark.read.option("header", "true").csv("hdfs://localhost:9000/datasets/WeatherForcast")

type(df)  #告诉你现在df是pyspark dataframe而不是我们平常用的pandas dataframe，如果你想把现在这个pys-df转为平常用的pan-df，就相当于是下载到本地，太过占用内存，所以不要！
#pyspark dataframe本来就是用来处理数据量比较大的数据的，pan-df只适用于小型数据

df_copy = df.alias("df_copy") #为了防止原始数据被覆盖

#df.show()
df_copy.show()

+----------+------+------+----------+--------------+-------+-----+------+------+-------+-------+----------+
|number_sta|   lat|   lon|height_sta|          date|     dd|   ff|precip|    hu|     td|      t|       psl|
+----------+------+------+----------+--------------+-------+-----+------+------+-------+-------+----------+
|  14066001|49.330|-0.430|     2.000|20160101 00:00|210.000|4.400| 0.000|91.000|278.450|279.850|      NULL|
|  14126001|49.150| 0.040|   125.000|20160101 00:00|   NULL| NULL| 0.000|99.000|278.350|278.450|      NULL|
|  14137001|49.180|-0.460|    67.000|20160101 00:00|220.000|0.600| 0.000|92.000|276.450|277.650|102360.000|
|  14216001|48.930|-0.150|   155.000|20160101 00:00|220.000|1.900| 0.000|95.000|278.250|278.950|      NULL|
|  14296001|48.800|-1.030|   339.000|20160101 00:00|   NULL| NULL| 0.000|  NULL|   NULL|278.350|      NULL|
|  14357002|48.930|-0.690|   223.000|20160101 00:00|   NULL| NULL| 0.000|  NULL|   NULL|277.650|      NULL|
|  14366002|49.170| 0.230|  

In [6]:
#打印输出每一列的数据类型，发现是string
column_types = df.dtypes
print(column_types)

[('number_sta', 'string'), ('lat', 'string'), ('lon', 'string'), ('height_sta', 'string'), ('date', 'string'), ('dd', 'string'), ('ff', 'string'), ('precip', 'string'), ('hu', 'string'), ('td', 'string'), ('t', 'string'), ('psl', 'string')]


In [7]:
#将df里所有列从string转为数值类型（浮点类型、整数类型）的数据（因为字符串不能算均值）

all_columns = df.columns

# 循环遍历每列并将其转换为数值类型
for column in all_columns:
    df = df.withColumn(column, df[column].cast(FloatType()))  # 使用 FloatType() 作为示例，你可以根据需要选择其他类型

# 打印结果
df.show() #

+-----------+------+------+----------+----+-----+----+------+----+------+------+--------+
| number_sta|   lat|   lon|height_sta|date|   dd|  ff|precip|  hu|    td|     t|     psl|
+-----------+------+------+----------+----+-----+----+------+----+------+------+--------+
|1.4066001E7| 49.33| -0.43|       2.0|NULL|210.0| 4.4|   0.0|91.0|278.45|279.85|    NULL|
|1.4126001E7| 49.15|  0.04|     125.0|NULL| NULL|NULL|   0.0|99.0|278.35|278.45|    NULL|
|1.4137001E7| 49.18| -0.46|      67.0|NULL|220.0| 0.6|   0.0|92.0|276.45|277.65|102360.0|
|1.4216001E7| 48.93| -0.15|     155.0|NULL|220.0| 1.9|   0.0|95.0|278.25|278.95|    NULL|
|1.4296001E7|  48.8| -1.03|     339.0|NULL| NULL|NULL|   0.0|NULL|  NULL|278.35|    NULL|
|1.4357002E7| 48.93| -0.69|     223.0|NULL| NULL|NULL|   0.0|NULL|  NULL|277.65|    NULL|
|1.4366002E7| 49.17|  0.23|      62.0|NULL| NULL|NULL|   0.0|NULL|  NULL|279.55|    NULL|
|1.4372001E7|49.102|-0.765|     184.0|NULL|230.0| 4.1|   0.0|92.0|278.05|279.25|    NULL|
|1.4501002

In [8]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import Imputer

In [9]:
#==============================1.mean value（Imputer）==============#

columns_to_fill = ["dd", "ff", "precip", "hu", "td", "t", "psl"] #需要被填充的列名

# 创建 Imputer 实例并设置输入输出列
imputer = Imputer(inputCols=columns_to_fill, outputCols=[f"{col}" for col in columns_to_fill])

# 训练 Imputer 模型
imputer_model = imputer.fit(df)

# 应用模型填充缺失值
df_imputed = imputer_model.transform(df)

# 打印结果
df_imputed.show()

+-----------+------+------+----------+----+---------+---------+-----------+---------+---------+--------+---------+
| number_sta|   lat|   lon|height_sta|date|       dd|       ff|     precip|       hu|       td|       t|      psl|
+-----------+------+------+----------+----+---------+---------+-----------+---------+---------+--------+---------+
|1.4066001E7| 49.33| -0.43|       2.0|NULL|    210.0|      4.4|        0.0|     91.0|   278.45|  279.85|101738.58|
|1.4126001E7| 49.15|  0.04|     125.0|NULL|186.64313|3.8138418|        0.0|     99.0|   278.35|  278.45|101738.58|
|1.4137001E7| 49.18| -0.46|      67.0|NULL|    220.0|      0.6|        0.0|     92.0|   276.45|  277.65| 102360.0|
|1.4216001E7| 48.93| -0.15|     155.0|NULL|    220.0|      1.9|        0.0|     95.0|   278.25|  278.95|101738.58|
|1.4296001E7|  48.8| -1.03|     339.0|NULL|186.64313|3.8138418|        0.0|80.639915|281.39838|  278.35|101738.58|
|1.4357002E7| 48.93| -0.69|     223.0|NULL|186.64313|3.8138418|        0.0|80.63

In [ ]:
for column in df.columns:
    df = df.withColumn(column, col(column).cast("string"))
df.show()

In [ ]:
selected_column = df_copy.select("number_sta")
df = df.withColumn("number_sta", lit(selected_column.first()["number_sta"]))
df.show()

In [ ]:
selected_column = df_copy.select("date")
df = df.withColumn("date", lit(selected_column.first()["date"]))
df.show()

In [ ]:
#===============================2.linear regression=======================================#

# columns_to_fill = ["dd", "ff", "precip", "hu", "td", "t", "psl"]

# # # 创建一个 VectorAssembler 将特征列合并为一个特征向量列
# # feature_columns = ["number_sta"]  # 选择作为特征的列
# # assembler = VectorAssembler(inputCols=feature_columns, outputCol="features")

# # # 训练一个线性回归模型，用于填充每个缺失值列
# # for column_to_fill in columns_to_fill:
# #     # 过滤出不含缺失值的数据用于训练
# #     training_data = df.filter(col(column_to_fill).isNotNull())

# #     # 创建特征向量
# #     df_assembled = assembler.transform(training_data)

# #     # 创建线性回归模型
# #     lr = LinearRegression(featuresCol="features", labelCol=column_to_fill, predictionCol=f"{column_to_fill}_predicted")

# #     # 训练模型
# #     model = lr.fit(df_assembled)

# #     # 使用模型预测缺失值列
# #     df_predicted = assembler.transform(df.filter(col(column_to_fill).isNull()))
# #     df_predicted = model.transform(df_predicted)

# #     # 将预测结果填充回原始 DataFrame
# #     df = df.join(df_predicted.select("number_sta", f"{column_to_fill}_predicted"), "number_sta", "left_outer")
# #     df = df.withColumn(column_to_fill, col(f"{column_to_fill}_predicted").cast(df[column_to_fill].dataType))


# df.show()